In [ ]:
# !pip3 install pandas numpy matplotlib seaborn
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
import scipy

In [ ]:
sns.set()

# Preprocess

In [ ]:
stdpso = pd.read_csv(f'data/raw/std_pso_{datetime.date.today()}.csv')
stdpso = stdpso[~stdpso.benchmark.isin([
#     'Deb 3 Function',
#     'Generalized Price 2 Function',
#     'Mishra 7 Function',
#     'Mishra 1 Function',
#     'Generalized Paviani Function',
])]

# Calculate standardised gbest \in [0, 1]
# min_gbest = stdpso.groupby(['benchmark', 'rep_num'])['gbest_fit'].transform('min')
# max_gbest = stdpso.groupby(['benchmark', 'rep_num'])['gbest_fit'].transform('max')
# range_gbest = max_gbest - min_gbest
# stdpso['norm_gbest'] = (stdpso['gbest_fit'] - min_gbest).div(range_gbest)

# Calculate log10_standardised_gbest
# stdpso['log10_norm_gbest'] = np.log10(stdpso['norm_gbest'])

responses = ['gbest_fit'] # , 'gdiversity', 'perc_oob', 'norm_gbest', 'log10_norm_gbest'
summ = stdpso.groupby(['benchmark', 'iter_num'])[responses] \
        .describe() \
        .loc[:,(slice(None),['mean', 'std', 'min', '50%', 'max'])] \
        .reset_index()
for multi_col in summ.columns:
    if multi_col[0] in responses:
        summ[(multi_col[0], 'mean_less_std')] = summ[(multi_col[0], 'mean')] - summ[(multi_col[0], 'std')] 
        summ[(multi_col[0], 'mean_plus_std')] = summ[(multi_col[0], 'mean')] + summ[(multi_col[0], 'std')] 
summ = summ.reindex(columns=sorted(summ.columns))
summ

Full data

In [ ]:
g = sns.FacetGrid(
    data=stdpso,
    col="benchmark", 
    col_wrap=3,
    height=3.5, 
    aspect=1.5,
    sharey=False,
#     palette="flare",
)
y = 'gbest_fit'
g.map_dataframe(
    sns.lineplot,
    x="iter_num",
    y=y,
#     estimator=None,
#     units='rep_num',
#     ci=None
)
g.add_legend()
g.set_titles(
    col_template="{col_name}", 
#     row_template="{row_name}"
)
g.set_axis_labels("Iteration Number", f"{y.replace('_', ' ').title()}")
# plt.savefig('../report/figs/gbest_fitness_standard_pso.pdf')

Partial data

# Optimise Control Parameters per Benchmark

In [ ]:
opt = pd.read_csv(f'data/raw/opt_{datetime.date.today()}.csv')
opt = opt[~opt.benchmark.isin([
    'Deb 3 Function',
    'Generalized Price 2 Function',
    'Mishra 7 Function',
#     'Mishra 1 Function',
    'Generalized Paviani Function',
])]

opt['c'] = np.round(opt['c1'] + opt['c2'], 4)
opt['w'] = np.round(opt['w'], 4)

## Heatmaps of Control Params

In [ ]:
opt_gb = opt[opt.iter_num == 4900].groupby(['benchmark', 'w', 'c'])['gbest_fit'].std().sort_values().reset_index()
opt_gb['log_gbest'] = np.log10(opt_gb['gbest_fit'])
# opt_gb

In [ ]:
def draw_heatmap(*args, **kwargs):
    data = kwargs.pop('data')
    d = data.pivot(columns=args[0], index=args[1], values=args[2])
    d = d.reindex(index=sorted(d.index)[::-1])
    d = d.reindex(columns=sorted(d.columns))
    ax = sns.heatmap(d, **kwargs)
    w_orig = np.arange(0, 1.1, 0.001)
    w = -(w_orig-0.5) + 0.5
    c = (24 - 24 * w * w) / (7 - 5 * w)
    plt.plot(
        (c * 1.3) + 0.5,
        (w_orig * 4.55) + 0.5,
        c='grey'
    )
    
g = sns.FacetGrid(opt_gb,
                  col='benchmark', 
                  col_wrap=3, 
                  palette="magma",
                  height=5)

g.map_dataframe(draw_heatmap, 'c', 'w', 'log_gbest',
#                 cbar=False,
                annot=True,
                yticklabels=True, 
                xticklabels=True)

g.set_titles(
    col_template="{col_name} ($\log_{{10}}$ scale)", 
)
g.set_axis_labels("$c=c_1+c_2$", "$w$")
# g.fig.subplots_adjust(top=0.85)
# g.fig.suptitle('Heatmap of Global Best Fitness values by Control Parameter\n $\log_{10}(fitness)$')
# plt.tight_layout()
plt.savefig('../report/figs/heatmaps.pdf')

## Best fitness value over time

In [ ]:
opt_ot = opt.groupby(['benchmark','w', 'c', 'iter_num']).gbest_fit.mean().reset_index()
opt_ot['log_gbest'] = np.log10(opt_ot['gbest_fit'])
opt_ot

In [ ]:
g = sns.FacetGrid(
    data=opt, 
    col="benchmark",
    hue='w',
    col_wrap=3,
    height=3.5, 
    aspect=1.5,
    sharey=False,
    palette="magma"
)
g.map(
    sns.lineplot, 
    "iter_num", 
    "perc_oob",
#     units='w',
#     estimator=None,
    ci=None
)
g.add_legend()
g.set_titles(
    col_template="{col_name}", 
#     row_template="{row_name}"
)
g.set_axis_labels("Iteration Number", "Global Best Fitness")
# g.fig.subplots_adjust(top=0.93)
# g.fig.suptitle('Global best fitness per function over time')
# plt.savefig('../report/figs/gbest_fitness_standard_pso.pdf')